# Laboratorio: Reti Neurali con Keras (parte 2)

**Programmazione di Applicazioni Data Intensive**  
Laurea in Ingegneria e Scienze Informatiche  
DISI - Università di Bologna, Cesena

Proff. Gianluca Moro, Roberto Pasolini  
`nome.cognome@unibo.it`

## Setup

- Importare le librerie necessarie per verificare il funzionamento
  - NB: utilizzeremo l'implementazione dell'API Keras interna a TensorFlow (usando il package `tensorflow.keras` invece di `keras`) per consentire facilmente l'esportazione del modello alla fine

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import tensorflow as tf

- Definire la seguente funzione per scaricare i file

In [2]:
import os
from urllib.request import urlretrieve
def download(file, url):
    if not os.path.isfile(file):
        urlretrieve(url, file)

## Ripasso: Reti neurali

- Una _rete neurale_ è un modello di apprendimento costituito da molteplici strati di nodi elementari
- Ciascun nodo è in pratica un modello di regressione, i cui input sono forniti dallo strato precedente e il cui output è passato a quello successivo
  - per modellare relazioni non lineari si applicano agli output dei nodi delle _funzioni di attivazione_, ad es. la funzione _ReLU_
- Tramite la _backpropagation_, i parametri (pesi e bias) di tutti i nodi sono addestrati congiuntamente per ottimizzare l'errore della rete tramite discesa gradiente stocastica
  - il training set è iterato molteplici volte (_epoche_), ad ogni iterazione le istanze vengono considerate a gruppi (_minibatch_)
- Ci sono molti aspetti configurabili (_iperparametri_) nella configurazione e nell'addestramento di una rete
  - struttura della rete: numero di strati, numero di nodi in ciascuno, funzione di attivazione, ...
  - addestramento: batch size, numero di epoche, ...

## Caso di Studio: Riconoscimento Attività Umane

- Gli smartphone moderni contengono diversi sensori di movimento, quali accelerometro e oscilloscopio
- È possibile raccogliendo dati da questi sensori risconoscere quale attività stia svolgendo una persona?
  - È in piedi fermo? Sta camminando? È seduto? ...
- Vediamo come addestrare una rete neurale a riconoscere l'attività svolta da una sequenza di letture dei sensori
- Tale rete può essere in seguito integrata in un'app per smartphone, ad es. per il tracking di attività fisica

## Dataset

- Utilizziamo un set di dati di letture da sensori diponibile online su https://archive.ics.uci.edu/ml/datasets/human+activity+recognition+using+smartphones
- Sono distinte **6 classi relative a diverse attività** o posizioni...
  - camminare, salire scale, scendere scale, seduti, in piedi, sdraiati
- ...svolte da 30 persone mentre indossavano uno smartphone
- 50 volte al secondo sono stati campionati **9 valori**
  - 3 sensori (accelerazione con e senza gravità, rotazione) per 3 assi (XYZ)
- Il dataset finale ha **10.299 finestre temporali** (tra training e test) di **128 istanti** l'una, a ciascuna finestra è associata una delle 6 classi

- Scarichiamo il dataset in formato ZIP dal Web...

In [3]:
download("HARDataset.zip", "https://archive.ics.uci.edu/ml/machine-learning-databases/00240/UCI%20HAR%20Dataset.zip")

- ...ed estraiamo i file

In [4]:
from zipfile import ZipFile
if not os.path.isdir("UCI HAR Dataset"):
    with ZipFile("HARDataset.zip") as zipf:
        zipf.extractall()

- I dati sono già divisi in due set `train` e `test`
- Nella directory `Inertial Signals` di ciascuno si trovano i file con i dati grezzi ottenuti dai sensori
- Sono considerati 3 diversi sensori:
  - `total_acc`: accelerazione (accelerometro)
  - `body_acc`: accelerazione senza la forza di gravità
  - `body_gyro`: rotazione (giroscopio)
- Per ogni sensore si considerano tre assi x, y, z come da figura:

![x da sinistra verso destra, y dal basso verso l'alto, z da dietro verso davanti](https://developer.android.com/images/axis_device.png)

- La seguente funzione carica tutti i dati descritti sopra dai file estratti

In [5]:
def load_dataset(prefix, sensors):
    def load_file(filepath):
        dataframe = pd.read_csv(filepath, header=None, delim_whitespace=True)
        return dataframe.values
    def load_dataset_group(group, prefix, sensors):
        filepath = prefix + group + "/Inertial Signals/"
        filenames = ["{}_{}_{}.txt".format(sensor, axis, group)
                     for sensor in sensors for axis in "xyz"]
        X_data = [load_file(filepath + name) for name in filenames]
        X = np.dstack(X_data)
        y = load_file(prefix + group + '/y_'+group+'.txt').ravel() - 1
        return X, y
    trainX, trainy = load_dataset_group('train', prefix, sensors)
    testX, testy = load_dataset_group('test', prefix, sensors)
    return trainX, trainy, testX, testy

- Invochiamo la funzione, specificando i sensori per cui vogliamo caricare i dati
  - si può eventualmente usare solo una parte dei sensori, ad es. per smartphone dotati solo di alcuni di essi

In [6]:
X_train, y_train, X_test, y_test = load_dataset(
    "UCI HAR Dataset/",
    ["total_acc", "body_acc", "body_gyro"]
)

- Otteniamo due dataset "train" e "test", costituiti rispettivamente da 7.352 e da 2.947 osservazioni (finestre temporali)
- Gli array `X_*` a 3 dimensioni (assi) contengono i valori campionati dai sensori
  - lungo l'asse 0 abbiamo le N finestre temporali
  - lungo l'asse 1 abbiamo i 128 istanti
  - lungo l'asse 2 abbiamo i 9 valori campionati per istante (3 sensori per 3 assi)
  - in pratica, il valore `[i,j,k]` è il valore di indice k campionato all'instante j nella finestra temporale i

In [7]:
X_train.shape

(7352, 128, 9)

In [8]:
X_test.shape

(2947, 128, 9)

- Gli array `y_*` a 1 dimensione contiene le etichette delle finestre temporali
  - le etichette sono comprese tra 0 e 5

In [9]:
y_train.shape

(7352,)

In [10]:
y_test.shape

(2947,)

- Carichiamo dal file `activity_labels.txt` i nomi delle attività riconosciute

In [11]:
with open("UCI HAR Dataset/activity_labels.txt", "rt") as f:
    labels = [line.split(" ")[1].strip() for line in f]

In [12]:
labels

['WALKING',
 'WALKING_UPSTAIRS',
 'WALKING_DOWNSTAIRS',
 'SITTING',
 'STANDING',
 'LAYING']

- Possiamo usarle per vedere la distribuzione di osservazioni delle diverse attività in training e test set

In [13]:
pd.Series(labels)[y_train].value_counts()

LAYING                1407
STANDING              1374
SITTING               1286
WALKING               1226
WALKING_UPSTAIRS      1073
WALKING_DOWNSTAIRS     986
dtype: int64

In [14]:
pd.Series(labels)[y_test].value_counts()

LAYING                537
STANDING              532
WALKING               496
SITTING               491
WALKING_UPSTAIRS      471
WALKING_DOWNSTAIRS    420
dtype: int64

- Codifichiamo le etichette (y) in vettori one-hot da usare come output atteso della rete

In [15]:
from tensorflow.keras.utils import to_categorical
yt_train = to_categorical(y_train)
yt_test = to_categorical(y_test)

- I valori in ingresso hanno già media vicina a 0 e dev. standard contenuta, non è necessario standardizzarli

In [16]:
X_train.mean((0, 1))

array([ 8.04749279e-01,  2.87554865e-02,  8.64980163e-02, -6.36303058e-04,
       -2.92296856e-04, -2.75299412e-04,  5.06464674e-04, -8.23780831e-04,
        1.12948439e-04])

In [17]:
X_train.std((0, 1))

array([0.41411195, 0.39099543, 0.35776881, 0.19484634, 0.12242748,
       0.10687881, 0.40681506, 0.38185432, 0.25574314])

- Dalle dimensioni degli array ricaviamo il numero di campioni per osservazione (128), di valori per campione (9) e di possibili classi (6)

In [18]:
n_timesteps = X_train.shape[1]
n_features = X_train.shape[2]
n_outputs = yt_train.shape[1]

- Costruiamo una prima rete per la classificazione con un singolo strato nascosto
- In input usiamo uno strato `Flatten` per convertire la matrice 128x9 con cui è rappresentata ciascuna osservazione in un vettore di 1.152 elementi
  - questo strato cambia solo la rappresentazione dei dati, non introduce nodi o parametri
  - con `input_shape` indichiamo la dimensione attesa della matrice
- Essendo un problema di classificazione, in output usiamo uno strato con attivazione softmax con 6 nodi, uno per classe

In [19]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
model = Sequential([
    Flatten(input_shape=(n_timesteps, n_features)),
    Dense(32, activation="relu"),
    Dense(n_outputs, activation="softmax")
])

- Vediamo la struttura della rete con la forma dell'output e il numero di parametri per ogni strato

In [20]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 1152)              0         
_________________________________________________________________
dense (Dense)                (None, 32)                36896     
_________________________________________________________________
dense_1 (Dense)              (None, 6)                 198       
Total params: 37,094
Trainable params: 37,094
Non-trainable params: 0
_________________________________________________________________


- Come visto le altre volte, compiliamo il modello specificando
  - di utilizzare l'algoritmo di ottimizzazione _Adam_ (variante della discesa gradiente stocastica)
  - di ottimizzare (minimizzandola) la _categorical cross entropy_, tanto più alta quanto più le probabilità date alle classi corrette si allontanano dal 100\%
  - di calcolare in parallelo anche l'accuratezza (percentuale di classificazioni corrrette)

In [21]:
model.compile(
    optimizer="adam",
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)

- Addestriamo quindi il modello con `fit`, specificando
  - il training set (input e relativi output attesi)
  - il numero di epoche di addestramento
  - la _batch size_, il numero di osservazioni (finestre temporali) in ciascun minibatch di addestramento

In [22]:
%time model.fit(X_train, yt_train, epochs=10, batch_size=20)

Epoch 1/10
368/368 [==============================] - 0s 919us/step - loss: 0.7217 - accuracy: 0.7458
Epoch 2/10
368/368 [==============================] - 0s 914us/step - loss: 0.3206 - accuracy: 0.8945
[...]
Epoch 9/10
368/368 [==============================] - 0s 914us/step - loss: 0.1120 - accuracy: 0.9566
Epoch 10/10
368/368 [==============================] - 0s 894us/step - loss: 0.1080 - accuracy: 0.9572
CPU times: user 4.57 s, sys: 284 ms, total: 4.85 s
Wall time: 3.84 s


- Usiamo _evaluate_ per calcolare sul validation set le stesse metriche di valutazione mostrate sul training set durante l'addestramento

In [23]:
model.evaluate(X_test, yt_test)

93/93 [==============================] - 0s 949us/step - loss: 0.3685 - accuracy: 0.8907


[0.36853569746017456, 0.8907363414764404]

- L'accuratezza (il secondo numero) assume valori indicativamente tra 85\% e 90\% (suscettibili di casualità)
- Salviamo questo modello in una variabile a parte per vedere successivamente come esportarlo...

In [24]:
model_to_export = model

- Possiamo aggiungere uno strato nascosto per rendere più accurata la rete

In [25]:
model = Sequential([
    Flatten(input_shape=(n_timesteps, n_features)),
    Dense(128, activation="relu"),
    Dense(64, activation="relu"),
    Dense(n_outputs, activation="softmax")
])

- Si noti che il numero di parametri da addestrare si alza sensibilmente...

In [26]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_1 (Flatten)          (None, 1152)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 128)               147584    
_________________________________________________________________
dense_3 (Dense)              (None, 64)                8256      
_________________________________________________________________
dense_4 (Dense)              (None, 6)                 390       
Total params: 156,230
Trainable params: 156,230
Non-trainable params: 0
_________________________________________________________________


In [27]:
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

In [28]:
%time model.fit(X_train, yt_train, epochs=10, batch_size=20)

Epoch 1/10
368/368 [==============================] - 0s 1ms/step - loss: 0.4349 - accuracy: 0.8436
Epoch 2/10
368/368 [==============================] - 1s 1ms/step - loss: 0.1641 - accuracy: 0.9372
[...]
Epoch 9/10
368/368 [==============================] - 0s 1ms/step - loss: 0.1189 - accuracy: 0.9518
Epoch 10/10
368/368 [==============================] - 1s 1ms/step - loss: 0.1100 - accuracy: 0.9555
CPU times: user 7.34 s, sys: 404 ms, total: 7.74 s
Wall time: 5.27 s


In [29]:
model.evaluate(X_test, yt_test)

93/93 [==============================] - 0s 903us/step - loss: 0.5236 - accuracy: 0.8789


[0.5235555768013, 0.8788598775863647]

## Reti convoluzionali

- Negli input delle reti neurali è spesso necessario **riconoscere dei pattern** che possono essere presenti **in diverse porzioni** dell'input
  - nel caso comune delle immagini, si vogliono riconoscere dei particolari indipendentemente dal punto in cui si trovano
  - nel caso di studio corrente, potremmo riconoscere delle sequenze temporali di valori che sono peculiari di attività specifiche
- Le reti _convoluzionali_ utilizzano strati con connessioni "locali" e pesi condivisi
  - ogni nodo riceve input solamente **da nodi vicini tra loro** nello strato inferiore, assumendo che corrispondano a **porzioni di spazio o di tempo**
  - **gli stessi pesi sono applicati a tutti i nodi**, in modo lo stesso pattern sia cercato sull'intero intervallo di spazio o di tempo analizzato

- Inseriamo all'inizio della rete uno strato `Conv1D` impostando il numero e la lunghezza dei pattern da cercare
  - ad esempio poniamo di cercare parallelamente 16 pattern con lunghezza di 15 passi temporali ciascuno
- L'output dello strato `Conv1D` sarà un array 2D che indica quali pattern sono stati individuati e in che punto dell'input
  - come sopra, applichiamo `Flatten` ad esso per ottenere un vettore lineare di nodi

In [30]:
from tensorflow.keras.layers import Conv1D
model = Sequential([
    Conv1D(16, 15, input_shape=(n_timesteps, n_features)),
    Flatten(),
    Dense(64, activation="relu"),
    Dense(n_outputs, activation="softmax")
])

- Dal sommario, vediamo che lo strato convoluzionale ha un numero di parametri nettamente inferiore rispetto ad un tipico strato denso
  - per ognuno dei 16 pattern abbiamo 15x9 pesi e un bias condivisi su 114 nodi, per un totale di 2.176 parametri
- Lo strato restituisce 114x16 valori, ovvero 16 pattern cercati nelle 114 (128-15+1) sequenze possibili di 15 valori su 128

In [31]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d (Conv1D)              (None, 114, 16)           2176      
_________________________________________________________________
flatten_2 (Flatten)          (None, 1824)              0         
_________________________________________________________________
dense_5 (Dense)              (None, 64)                116800    
_________________________________________________________________
dense_6 (Dense)              (None, 6)                 390       
Total params: 119,366
Trainable params: 119,366
Non-trainable params: 0
_________________________________________________________________


In [32]:
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

In [33]:
%time model.fit(X_train, yt_train, epochs=10, batch_size=20)

Epoch 1/10
368/368 [==============================] - 1s 2ms/step - loss: 0.5474 - accuracy: 0.8071
Epoch 2/10
368/368 [==============================] - 1s 2ms/step - loss: 0.2083 - accuracy: 0.9233
[...]
Epoch 9/10
368/368 [==============================] - 1s 2ms/step - loss: 0.1074 - accuracy: 0.9554
Epoch 10/10
368/368 [==============================] - 1s 2ms/step - loss: 0.0965 - accuracy: 0.9592
CPU times: user 13 s, sys: 532 ms, total: 13.5 s
Wall time: 7.53 s


In [34]:
model.evaluate(X_test, yt_test)

93/93 [==============================] - 0s 1ms/step - loss: 0.4774 - accuracy: 0.9033


[0.4774245023727417, 0.9032914638519287]

## Reti ricorrenti

- Al contrario delle reti viste finora, una rete _ricorrente_ contiene connessioni cicliche tra nodi
- A queste reti l'input **è fornito sequenzialmente** in più passi temporali
  - nel nostro caso, si immagini di fornire le 128 letture dei sensori una dopo l'altra invece che in blocco
- Tramite le connessioni cicliche, **la rete mantiene uno stato** da un passo all'altro
- Per usare una rete ricorrente, nei dati di addestramento e test deve essere presente una dimensione temporale
  - come nel caso di studio corrente, dove ogni osservazione è una sequenza di 128 campioni
- Le reti ricorrenti possono potenzialmente riconoscere correlazioni tra dati forniti in passi temporali diversi, anche distanti

- Keras fornisce diversi tipi di strati ricorrenti, tra cui le _Gated Recurrent Unit_ (GRU)
  - ad ogni passo temporale $t$ lo strato GRU calcola un output di N valori $h(t)$ in funzione sia dell'input attuale $x(t)$ che dell'output precedente $h(t-1)$
- In addestramento e test, dobbiamo fornire le osservazioni complete di tutti gli istanti temporali
  - per convenzione Keras tratta una matrice di dimensioni L×M×N come L sequenze di durata M di vettori di misura N
  - nel nostro caso, L sequenze di 128 campioni di 9 valori
- Di default, solo l'output di GRU all'ultimo passo temporale è considerato per determinare l'output dato dalla rete ad ogni osservazione

In [35]:
from tensorflow.keras.layers import GRU
model = Sequential([
    GRU(64, activation="relu", input_shape=(n_timesteps, n_features)),
    Dense(n_outputs, activation="softmax")
])

- Anche in questo caso il numero di parametri è molto inferiore rispetto ad un MLP ordinario, in quanto gli stessi pesi vengono riutilizzati attraverso i 128 passaggi temporali

In [36]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
gru (GRU)                    (None, 64)                14400     
_________________________________________________________________
dense_7 (Dense)              (None, 6)                 390       
Total params: 14,790
Trainable params: 14,790
Non-trainable params: 0
_________________________________________________________________


In [37]:
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

- Il tempo per l'addestramento aumenta, in quanto l'errore su ciascuna osservazione va derivato attraverso i 128 passi temporali (_backpropagation through time_)

In [38]:
%time model.fit(X_train, yt_train, epochs=5, batch_size=100)

Epoch 1/5
74/74 [==============================] - 3s 41ms/step - loss: 1.5351 - accuracy: 0.4452
Epoch 2/5
74/74 [==============================] - 3s 40ms/step - loss: 1.1608 - accuracy: 0.6066
Epoch 3/5
74/74 [==============================] - 3s 42ms/step - loss: 0.7020 - accuracy: 0.7014
Epoch 4/5
74/74 [==============================] - 3s 39ms/step - loss: 0.5566 - accuracy: 0.7675
Epoch 5/5
74/74 [==============================] - 3s 39ms/step - loss: 0.4402 - accuracy: 0.8251
CPU times: user 32.1 s, sys: 1.73 s, total: 33.9 s
Wall time: 15.9 s


In [39]:
model.evaluate(X_test, yt_test)

93/93 [==============================] - 1s 7ms/step - loss: 0.6135 - accuracy: 0.7387


[0.6135448813438416, 0.7387173175811768]

## Deployment del modello con TensorFlow Lite

- TensorFlow Lite è un framework per il deep learning destinato a dispositivi mobili ed embedded
- Si usa per eseguire inferenze (stime e predizioni) su modelli che vengono caricati già addestrati sul dispositivo
  - l'inferenza richiede generalmente molte meno risorse rispetto all'addestramento
- Sul Web esistono diversi modelli preaddestrati per diversi task
  - riconoscimento immagini, natural language processing, ...
- Possiamo in aggiunta esportare i modelli addestrati con TensorFlow (anche tramite Keras)
- Nel nostro caso di studio, possiamo esportare il modello che riconosce le attività, per poi utilizzarlo ad es. all'interno di un'app mobile

## Esportazione del modello

- Creiamo un oggetto `TFLiteConverter` passando il modello Keras addestrato
  - NB: funziona solo su modelli che usano il package `tensorflow.keras`

In [40]:
converter = tf.lite.TFLiteConverter.from_keras_model(model_to_export)

- Utilizziamone il metodo `convert` per ottenere la rappresentazione binaria
  - NB: eseguendo da Jupyter, si ha errore se TensorFlow è installato in un ambiente virtuale diverso da quello di Jupyter

In [41]:
tflite_model = bytes()
#tflite_model = converter.convert()

- Esportiamo quindi tale rappresentazione in un file

In [42]:
with tf.io.gfile.GFile("model.tflite", "wb") as f:
    f.write(tflite_model)

## Uso all'interno di un'app Android

- Vediamo in breve come si integra il modello esportato in un'app Android
- Per iniziare, dichiariamo TensorFlow Lite come dipendenza nel file `build.gradle` del progetto

```groovy
repositories {
  // ... altre repository ...
  maven {
    url 'https://google.bintray.com/tensorflow'
  }
}
dependencies {
  // ... altre dipendenze ...
  implementation 'org.tensorflow:tensorflow-lite:+'
}
```

- Un modello TF Lite è rappresentato a run-time da un oggetto `Interpreter`
- Creiamo tale oggetto passando un riferimento al file o direttamente un buffer con i dati
  - si può trattare di un file salvato nel file system, es. scaricato da Web
  - si può anche integrare il modello nell'app (nel file APK), configurandolo in modo che non venga compresso
- L'oggetto va generalmente creato all'apertura dell'app (`Activity.onCreate`) e chiuso col metodo `close` alla terminazione (`Activity.onDestroy`)

```java
import org.tensorflow.lite.Interpreter;
...
File modelFile = new File("/path/to/model.tflite");
Interpreter model = new Interpreter(modelFile);
```

- Per usare il modello vanno allocati per i suoi input e output dei buffer delle giuste dimensioni, che possono essere
  - degli array Java (come da esempio sotto)
  - oggetti `ByteBuffer` (meno immediati da usare ma più efficienti)

```java
// array 2D dove inserire l'input da passare alla rete
float[][] inputBuffer = new float[seqLength][inputSize];
// array in cui verrà scritto l'output della rete
float[] outputBuffer = new int[numClasses];
```

- Ogni volta che si vuole eseguire l'inferenza si usa il metodo `run` dell'`Interpreter`

```java
model.run(inputBuffer, outputBuffer);
```

- Nel nostro caso di studio, possiamo usare l'API `SensorManager` di Android per eseguire una callback ogni volta che arrivano dati dai sensori
- La callback andrà a riportare i dati ricevuti nel buffer di input e ad invocare il modello quando questo viene riempito

```java
void feedSensorData(int sensor, SensorEvent event) {
  // determina a quale passo temporale
  // si riferiscono i dati ricevuti
  int timestep = ...;
  // copia i dati del sensore nel buffer
  System.arraycopy(event.values, 0, inputBuffer[inputFillStep],
      NUM_AXES * sensor, NUM_AXES);
  // se sono arrivato al termine del buffer...
  boolean bufferIsFull = ...;
  if (bufferIsFull) {
    // esegui l'inferenza
    inferActivity();
    // azzera il buffer di input
    resetInputBuffer();
  }
}
```

- Quando il buffer è pieno, si passa il contenuto al modello, si ottengono le probabilità delle classi e si verifica qual è la più probabile

```java
int argmax(float[] values) {
  /** Restituisci indice del valore maggiore. */ ...
}

void inferActivity() {
  // eseguo l’inferenza con i dati raccolti
  model.run(inputBuffer, outputBuffer);
  // verifico la classe con probabilità maggiore
  int activityClass = argmax(outputBuffer);
  // invoco una callback (ad es. per aggiornare la GUI)
  callback.activityInferred(activityClass);
}
```